## Import Libraries

In [1]:
import requests
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Documentation about columns:  https://dev.socrata.com/foundry/data.cdc.gov/n8mc-b4w4

## Endpoint Generation and Data Retrieval

In [2]:
# Establishing base endpoint url and dummy variables that can be changed

endpoint = 'https://data.cdc.gov/resource/n8mc-b4w4.json'
case_month = '2022-07'
limit = '50000'
res_state = 'CO'   #  Pass in states_2letter_list here <-----------

# Below generates a list of two-letter state codes from a website

url = 'https://www.ssa.gov/international/coc-docs/states.html'
states_table = pd.read_html(url)
states_df = pd.DataFrame(data = states_table[0])
states_2letter_list = states_df[1].to_list()
df = pd.DataFrame()
# This is the function that will retrieve the data from the cdc.gov Api

def get_data(case_month, limit, res_state):
    criteria = f'case_month={case_month}&$limit={limit}&res_state={res_state}'
    data = requests.get(endpoint+'?'+criteria)
    data_list = data.json()
    df_data = pd.DataFrame(data_list)
    return df_data

In [ ]:
all_data = pd.DataFrame()
for x in range (0,len(states_2letter_list)-1):
    df = get_data(case_month,limit, states_2letter_list[x])
    all_data = pd.concat([all_data,df],ignore_index=True)
all_data.info

## Read from the Dataframe/CSV for Columns & Datatypes

In [10]:
col_list = all_data.columns
col_list

Index(['case_month', 'res_state', 'state_fips_code', 'res_county',
       'county_fips_code', 'age_group', 'sex', 'race', 'ethnicity', 'process',
       'exposure_yn', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn',
       'death_yn', 'underlying_conditions_yn', 'case_onset_interval',
       'case_positive_specimen'],
      dtype='object')

In [11]:
type(col_list)

pandas.core.indexes.base.Index

In [8]:
dtypes_list = all_data.dtypes
dtypes_list

case_month                  object
res_state                   object
state_fips_code             object
res_county                  object
county_fips_code            object
age_group                   object
sex                         object
race                        object
ethnicity                   object
process                     object
exposure_yn                 object
current_status              object
symptom_status              object
hosp_yn                     object
icu_yn                      object
death_yn                    object
underlying_conditions_yn    object
case_onset_interval         object
case_positive_specimen      object
dtype: object

In [12]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

In [16]:
columns_string = ", ".join("{} {}".format(n,d) for (n,d) in zip(col_list, dtypes_list.replace(replacements)))
columns_string

'case_month varchar, res_state varchar, state_fips_code varchar, res_county varchar, county_fips_code varchar, age_group varchar, sex varchar, race varchar, ethnicity varchar, process varchar, exposure_yn varchar, current_status varchar, symptom_status varchar, hosp_yn varchar, icu_yn varchar, death_yn varchar, underlying_conditions_yn varchar, case_onset_interval varchar, case_positive_specimen varchar'

## Connect to PostgreSQL Database

In [17]:
all_data.to_csv('cov_2022_07_data_2.csv', header=all_data.columns, index=False, encoding='utf-8')

In [18]:
my_file = open('cov_2022_07_data_2.csv')
print('file opened in memory')

file opened in memory


In [22]:
try:
    # Open the connection
    conn = psycopg2.connect(host="localhost",
    database="postgres",user="postgres",password="password",port ="5432")
    
    # Open the cursor
    cur = conn.cursor()
    
    create_script = '''
    CREATE TABLE IF NOT EXISTS covid_data (%s)
    ''' % (columns_string)
    
    upload_csv_script = '''
    COPY covid_data
    FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
    '''
    
    cur.execute(create_script)
    cur.execute('TRUNCATE TABLE covid_data')
    cur.copy_expert(sql=upload_csv_script, file=my_file)
    
    # Commit the script to finalize the changes
    conn.commit()
    
    # Close the connection & cursor
    cur.close()
    conn.close()
except Exception as error:
    print(error)

## Now we have our data in our PostgreSQL Database!